In [3]:
import numpy as np
from scipy.optimize import minimize

# Función objetivo: Rosenbrock
def rosenbrock(x):
    return sum(100.0*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

# Generación de población inicial
def generate_initial_population(pop_size, dim, bounds):
    return np.random.rand(pop_size, dim) * (bounds[:,1] - bounds[:,0]) + bounds[:,0]

# Evaluación de la población
def evaluate_population(population, objective_function):
    return np.array([objective_function(ind) for ind in population])

# Selección de los mejores individuos
def select_best_individuals(population, fitness, num_best):
    indices = np.argsort(fitness)
    return population[indices[:num_best]]

# Cruce de dos individuos
def crossover(ind1, ind2):
    alpha = np.random.rand()
    return alpha * ind1 + (1 - alpha) * ind2

# Mutación de un individuo
def mutate(individual, mutation_rate, bounds):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            individual[i] += np.random.normal(0, 1) * (bounds[i,1] - bounds[i,0]) * 0.1
            individual[i] = np.clip(individual[i], bounds[i,0], bounds[i,1])
    return individual

# Optimización Híbrida GA-Nelder-Mead
def hybrid_ga(objective_function, bounds, pop_size, generations, num_best, mutation_rate):
    dim = bounds.shape[0]
    population = generate_initial_population(pop_size, dim, bounds)
    
    for generation in range(generations):
        fitness = evaluate_population(population, objective_function)
        best_individuals = select_best_individuals(population, fitness, num_best)
        
        # Generación de nueva población
        new_population = []
        while len(new_population) < pop_size:
            parents = best_individuals[np.random.choice(num_best, 2, replace=False)]
            child = crossover(parents[0], parents[1])
            child = mutate(child, mutation_rate, bounds)
            new_population.append(child)
        population = np.array(new_population)
        
    # Optimización local Nelder-Mead en las mejores soluciones
    final_fitness = evaluate_population(population, objective_function)
    best_individual = population[np.argmin(final_fitness)]
    result = minimize(objective_function, best_individual, method='Nelder-Mead')
    
    return result

# Parámetros
bounds = np.array([[-2, 2], [-2, 2]])
pop_size = 50
generations = 100
num_best = 10
mutation_rate = 0.05

# Ejecución del algoritmo híbrido
result = hybrid_ga(rosenbrock, bounds, pop_size, generations, num_best, mutation_rate)
print(f"Parámetros optimizados: {result.x}")
print(f"Valor mínimo: {result.fun}")


Parámetros optimizados: [1.00001521 1.00003105]
Valor mínimo: 2.7248397183202864e-10
